<a href="https://colab.research.google.com/github/dhyannn/Deep-learning/blob/main/2348507_DLLab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Deep Neural Network Model to Predict Customer Churn Rates

Tasked with developing a deep neural network (DNN) model to predict customer churn for a telecommunications company. The dataset provided contains various features such as customer demographics, usage patterns, and service subscription details. Objective is to implement dropout, layer-wise dropout, and Monte Carlo dropout techniques in the DNN architecture to assess their impact on model performance and generalization.

In [1]:
import pandas as pd
from tensorflow import keras
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tensorflow.keras.layers import  Dense,Dropout

In [2]:
Data = pd.read_csv('/content/IT_customer_churn.csv')
Data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [3]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


**Inference:** There is no null values in the dataset

In [4]:
Data.shape

(7043, 20)

# **Data Preprocessing**

Converting object datatype to numeric

In [6]:
cat_cols = Data.select_dtypes(include=['object']).columns.tolist()
le = LabelEncoder()
for col in cat_cols:
    Data[col] = le.fit_transform(Data[col])

In [8]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   int64  
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   int64  
 3   Dependents        7043 non-null   int64  
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   int64  
 6   MultipleLines     7043 non-null   int64  
 7   InternetService   7043 non-null   int64  
 8   OnlineSecurity    7043 non-null   int64  
 9   OnlineBackup      7043 non-null   int64  
 10  DeviceProtection  7043 non-null   int64  
 11  TechSupport       7043 non-null   int64  
 12  StreamingTV       7043 non-null   int64  
 13  StreamingMovies   7043 non-null   int64  
 14  Contract          7043 non-null   int64  
 15  PaperlessBilling  7043 non-null   int64  
 16  PaymentMethod     7043 non-null   int64  


Splitting features and target variable

In [9]:
x = Data.drop('Churn',axis=1)
y = Data['Churn']

Splitting the data

In [10]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

# **Basic DNN architecture**

In [11]:
model = Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')

])

Compiling the model

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Training the baseline model

In [13]:
train = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# **Implementing the dropout model**

In [14]:
dropoutModel = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

Compiling the model

In [15]:
dropoutModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Train the dropout model

In [16]:
dropoutModel= dropoutModel.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# **Implementing Layer wise dropout model**

In [17]:
layerwiseDropoutModel = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])


Compiling the layer wise dropout model

In [18]:
layerwiseDropoutModel.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

Training the layer-wise dropout model

In [19]:
layerwiseDropout = layerwiseDropoutModel.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), verbose=0)

# **Implementing monte carlo dropout model**

In [20]:
class MCDropout(tensorflow.keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

mcDropoutModel = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    MCDropout(0.5),
    Dense(32, activation='relu'),
    MCDropout(0.5),
    Dense(1, activation='sigmoid')
])

Compiling the model

In [21]:
mcDropoutModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Training the monte carlo dropout model

In [22]:
mcDropout = mcDropoutModel.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), verbose=0)

Evaluating the model

In [23]:
def evaluateModel(model, X_test, y_test, threshold=0.5):
    y_pred_prob = model.predict_classes(X_test)
    y_pred = (y_pred_prob > threshold).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    return accuracy, f1, roc_auc

In [25]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def evaluateModel(model, X_test, y_test, threshold=0.5):
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > threshold).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    return accuracy, f1, roc_auc